# Exploring the Method of Increments with QEMIST Cloud and Tangelo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/goodchemistryco/Tangelo/blob/develop/examples/mifno.ipynb)

In this notebook, we illustrate how users can leverage QEMIST Cloud and Tangelo to explore the impact of quantum computing on problems tackled with the MI-FNO problem decomposition technique.

You do not need to have the `qemist-client` python package installed to run this notebook: only `tangelo` is required. For more information about [QEMIST Cloud](https://goodchemistry.com/qemist-cloud/) (installation, features, issues, `qemist_client` API...), please refer to the [QEMIST Cloud documentation](https://alpha-api.qemist.cloud/#/docs) or contact their development team.

The first section provides a high-level description of the MI-FNO approach. The second one briefly shows how QEMIST Cloud can apply this approach to a usecase, and provide reference results computed with high-accuracy classical solvers. We then focus on the API provided in Tangelo allowing users to combine the MI-FNO performed by QEMIST Cloud and any quantum workflow written with Tangelo.

This notebook assumes that you already have installed Tangelo in your Python environment, or have updated your Python path so that the imports can be resolved. If not, executing the cell below installs the minimal requirements for this notebook.

In [1]:
# Installation of tangelo if not already installed.
try:
    import tangelo
except ModuleNotFoundError:
    !pip install git+https://github.com/goodchemistryco/Tangelo.git@develop --quiet

# Download the data folder at https://github.com/goodchemistryco/Tangelo/tree/develop/examples/data
import os
if not os.path.isdir("data"):
    !sudo apt install subversion
    !svn checkout https://github.com/goodchemistryco/Tangelo/branches/develop/examples/data

## Use case

Our use case here is the hydrogen fluoride (HF) system defined below, using the `sto-3g` basis, chosen for simplicity. However, the MI-FNO method can be used to tackle much larger systems.

In [2]:
HF_coordinates = """
    F    0.0000   0.0000   0.0000
    H    0.0000   0.0000   0.9168
"""
basis = "sto-3g"
charge = 0 
spin = 0

## What is MI-FNO ?

The method of increment (MI) expresses the electron correlation energy of a molecular system as a truncated many-body expansion in terms of orbitals, atoms, molecules, or fragments. The electron correlation of the system is expanded in terms of occupied orbitals, and MI is employed to systematically reduce the occupied orbital space. At the same time, the virtual orbital space is reduced based on the frozen natural orbitals (FNO), which are obtained using a one-particle density matrix from second-order, many-body perturbation theory. In this way, a method referred to as the MI-FNO approach is available for the systematic reduction of both the occupied space and the virtual space in quantum chemistry simulations. 

MI was first introduced in quantum chemistry by Nesbet ([Phys. Rev. 1967, 155, 51](https://doi.org/10.1103/PhysRev.155.51), [Phys. Rev. 1967, 155, 56](https://doi.org/10.1103/PhysRev.155.56) and [Phys. Rev. 1968, 175, 2](https://doi.org/10.1103/PhysRev.175.2)), is based upon the n-body Bethe–Goldstone expansion ([Proc. R. Soc. A, 1957, 238, 551](https://doi.org/10.1098/rspa.1957.0017)) of the correlation energy of a molecule. The correlation energy ($E_c$), defined as the difference between the exact ($E_{\text{exact}}$) and the Hartree–Fock (mean-field) energy ($E_{\text{HF}}$), can be expanded as

$$
\begin{align*}
E_c &= E_{\text{exact}} - E_{\text{HF}} \\
&= \sum_i \epsilon_i + \sum_{i>j} \epsilon_{ij} + \sum_{i>j>k} \epsilon_{ijk} + \sum_{i>j>k>l} \epsilon_{ijkl} + \dots
\end{align*}
$$

where $\epsilon_i$, $\epsilon_{ij}$, $\epsilon_{ijk}$, and $\epsilon_{ijkl}$ are, respectively, the one-, two-, three-, and four-body increments (expansions) defined as

$$
\begin{align*}
\epsilon_i &= E_c(i) \\
\epsilon_{ij} &= E_c(ij) - \epsilon_i - \epsilon_j \\
\epsilon_{ijk} &= E_c(ijk) - \epsilon_{ij} - \epsilon_{ik} - \epsilon_{jk} - \epsilon_{i} - \epsilon_{j} - \epsilon_{k} \\
\epsilon_{ijkl} &= E_c(ijkl) - \epsilon_{ijk} - \epsilon_{ijl} - \epsilon_{jkl} - \dots \\
&\vdots
\end{align*}
$$

The following figure, taken from [J. Chem. Phys. 2021, 155, 034110](https://doi.org/10.1063/5.0054647), illustrates this problem decomposition scheme in terms of 1-body and many-body interactions. On each subproblem, an FNO truncation
is applied to reduce their virtual space. The subproblems resulting from the MI-FNO reduction can then be solved by any algorithm, including quantum algorithms such as the phase estimation algorithm and the variational quantum eigensolver (VQE), to predict the correlation energies of a molecular system.


<div>
<img src="img/mifno.png" width="600"/>
</div>

This problem decomposition pipeline is available in QEMIST Cloud. In this notebook, we illustrate how to export MI-FNO fragment data computed in QEMIST Cloud, and import it in Tangelo for further treatment, such as using quantum solvers.

## Performing MI-FNO calculations with QEMIST Cloud

[QEMIST Cloud](https://goodchemistry.com/qemist-cloud/) is an engine that enables faster, more accurate, and scalable ways to perform computational chemistry simulations. This platform leverages easily and readily accessible computers on the cloud to perform chemistry simulations that were previously intractable even on expensive, high-performance computing environments.

In order to leverage this platform to perform the MI-FNO calculations, subscribing to the services and installing the `qemist_client` python package is necessary. This notebook does not require either of these things: we provide the code snippet used to generate the problem decomposition results, and use those as pre-computed values for the rest of the notebook.

For more information about QEMIST Cloud (installation, features, issues, `qemist_client` API...), please refer to the [QEMIST Cloud documentation](https://alpha-api.qemist.cloud/#/docs) or contact their development team.

In the script below, each fragment's virtual space is truncated to keep only the virtual orbitals with the highest occupation number. MI-FNO is paired with the Heath-Bath Configuration Interaction (HBCI) classical solver.

```python
import os
os.environ['QEMIST_PROJECT_ID'] = "your_project_id_string"
os.environ['QEMIST_AUTH_TOKEN'] = "your_qemist_authentication_token"

from qemist_client.molecule import Molecule
from qemist_client.problem_decomposition import IncrementalDecomposition
from qemist_client.problem_reduction import FNO
from qemist_client.electronic_structure_solvers import HBCI
from qemist_client.util import get_results

HF_coordinates = """
    F    0.0000   0.0000   0.0000
    H    0.0000   0.0000   0.9168
"""
basis = "sto-3g"
charge = 0
spin = 0

# Creating the QEMIST molecule object.
HF_mol = Molecule(HF_coordinates, basis=basis, 
                  charge=charge, spin=spin)

# Defining the solvers, Heath-Bath Configuration Interaction (HBCI),
# Frozen Natural Orbitals (FNO), and the problem decomposition approach,
# Method of Increments (MI).
hbci_solver = HBCI()
fno = FNO(hbci_solver, export_fragment_data=True)
mi_solver = IncrementalDecomposition(solver=fno, truncation_order=3)

# Submitting the problem to the cloud.
HF_handle = mi_solver.simulate(molecule=HF_mol)
print(f"\nHF handle: {HF_handle}")

# Retrieving the results.
HF_result = get_results(HF_handle)
print(HF_result)
```

The `HF_result` python dictionary can be saved in a json file (for further treatment in Tangelo or for archiving purposes) with the following code.

```python
import json
with open("your_file_path", "w") as f:
    json.dump(HF_result, f)

for n__trunc, frags in HF_result["subproblem_data"].items():
    for frag_id, frag in frags.items():
        prob_handle = frag["problem_handle"]

        if problem_handle:
            frag_res = get_results(prob_handle)

            with open(f"your_folder_path"/{prob_handle}.json", "w") as f:
                json.dump(frag_res, f)
```

## Combining quantum algorithms with MI-FNO using Tangelo

In the future, pairing problem decomposition techniques such as MI-FNO with quantum solvers running in the cloud could be streamlined in QEMIST Cloud, which could directly call the solvers available in Tangelo.

But we think quantum hardware and quantum algorithm development has a bit of a way to go before this can be seen as a practical approach. However, we can manually explore how to combine MI-FNO with quantum solvers right now, in order to explore use cases intractable otherwise: maybe our discoveries can contribute in advancing the field, and bring that future a bit sooner.

This section illustrates how Tangelo can retrieve the results from a MI-FNO job run in QEMIST Cloud, and enable our quantum explorations.

### 1. Importing results from QEMIST Cloud

In Tangelo, the `MIFNOHelper` class facilitates importing the results generated by QEMIST Cloud, in our case contained in the `HF_result` object.

In [3]:
from tangelo.problem_decomposition import MIFNOHelper

`MIFNOHelper` can either directly take a "results" object in json format produced by QEMIST Cloud (using `results_object=...`), or the path to a file containing that object (using `json_file=...`). In our example, let's just assume `HF_results` has been saved in `./data/HF_MIFNO_trunc2.json` and let's see what information has been retrieved.

In [4]:
fno_fragments = MIFNOHelper(mi_json_file="./data/HF_STO3G_MIFNO_HBCI.json", fno_json_folder="./data")
print(fno_fragments)

(All the energy values are in hartree)
Total MI-FNO energy = -98.59655271223816
Correlation energy = -0.025795120623911316
Mean-field energy = -98.57075759161424
            epsilon     problem_handle  energy_total  correction  \
(1,)      -0.001326  65652562505002413    -98.572083         0.0   
(2,)      -0.019846  10837505518890413    -98.590604         0.0   
(3,)      -0.000862  31462118862908845    -98.571619         0.0   
(4,)      -0.000756  49501973643554221    -98.571513         0.0   
(1, 2)    -0.003024  65302015864237516    -98.594953         0.0   
(1, 3)     0.000085  53405772498103756    -98.572860         0.0   
(1, 4)     0.000142  13411621153304012    -98.572697         0.0   
(2, 3)    -0.000845  22700772126125516    -98.592310         0.0   
(2, 4)     0.000124  27549614109650380    -98.591235         0.0   
(3, 4)     0.000078  23150438022147532    -98.572297         0.0   
(1, 2, 3)  0.000466  30910159730798073    -98.596109         0.0   
(1, 2, 4) -0.000016  1

It parsed information contained in the "result blob" from QEMIST Cloud. We see that there is information about the whole system, including the mean-field energy and the total energy, as well as how each of the fragment -denoted by a tuple of integers- contributed to the correlation energy.

Although not displayed here, the object also contains information about the frozen orbitals used for each of these fragments ("frozen lists").

### 2. Importing molecular coefficients files

In order to build a `FermionOperator` or `QubitOperator` object compatible with our quantum algorithms, we can retrieve the MO coeffs exported by QEMIST Cloud. The "result blob" contains a URL where these are located, for each fragment: the `retrieve_mo_coeff` method downloads them in the target directory provided by the user, and then loads the MO coeffs into the `MIFNOHelper` object.

- The default target folder is the folder where the user script is executed.
- If the user does not provide a path to a valid existing directory, the call returns an error.
- If the files already exists in the target directory, they are not re-downloaded. The method keeps the user informed of progress as it is downloading files, otherwise it prints nothing.

In [5]:
fno_fragments.retrieve_mo_coeff("./data")

### 3. Reconstructing a fragment Hamiltonian

We can use the MO coeffs to modify the molecular integrals, in order to take into account the FNO localization, using our "frozen lists".

The `compute_fermionoperator` method handles the frozen orbitals for each fragment, and produces a `FermionOperator`. We are free to use the qubit mapping of our choice to produce a `QubitOperator` object, as input for a quantum algorithm.

In [6]:
from tangelo import SecondQuantizedMolecule
from tangelo.toolboxes.operators import count_qubits
from tangelo.toolboxes.qubit_mappings.mapping_transform import fermion_to_qubit_mapping

# Needed to compute the molecular integrals.
mol = SecondQuantizedMolecule(HF_coordinates, q=charge, spin=spin, basis=basis)

# Selection of a fragment and fetching the number of electrons and spinorbitals.
selected_fragment = "(1,)"
n_electrons, n_spinorbitals = fno_fragments.n_electrons_spinorbs(selected_fragment)

# Computing the related FermionOperator.
ferm_op = fno_fragments.compute_fermionoperator(mol, selected_fragment)

# Transformation of the FermionOperator to a QubitOperator.
qu_op = fermion_to_qubit_mapping(ferm_op, mapping="JW", up_then_down=False)

print(f"Fragment {selected_fragment} mapped to {count_qubits(qu_op)} qubits.")

Fragment (1,) mapped to 4 qubits.


### 4. Do your thing !

Now that we are able to construct `FermionOperator` and `QubitOperator` objects representing the subproblems defined by our fragments, we are free to throw them at any algorithm available in Tangelo. You could even decide to use your own custom workflow instead. Each fragment that is relevant to you could be solved with a different quantum solver, if you wish to do so. 

Below, we treat a given fragment with Quantum Imaginary Time Evolution (QITE) to obtain its energy. We made sure to enter values for `n_electrons` and `n_spinorbitals` that are consistent with our MI-FNO parameters.

This code cell can take several minutes to execute. After its execution, the value of `qite.final_energy` should be about `-98.57208`.

In [7]:
from tangelo.algorithms.projective import QITESolver
from tangelo.toolboxes.operators import qubitop_to_qubitham

qubit_ham = qubitop_to_qubitham(qu_op, mapping="JW", up_then_down=False)

qite = QITESolver({"qubit_hamiltonian": qubit_ham, "n_electrons": n_electrons, "n_spinorbitals": n_spinorbitals})
qite.build()
qite.simulate()

Iteration 1 of QITE with starting energy -98.57075759161438
Iteration 2 of QITE with starting energy -98.57113742411626
Iteration 3 of QITE with starting energy -98.571407878036
Iteration 4 of QITE with starting energy -98.57160051305051
Iteration 5 of QITE with starting energy -98.57173778008789
Iteration 6 of QITE with starting energy -98.57183564786958
Iteration 7 of QITE with starting energy -98.57190547376783
Iteration 8 of QITE with starting energy -98.57195533549731
Iteration 9 of QITE with starting energy -98.57199097857621
Iteration 10 of QITE with starting energy -98.57201649020138
Iteration 11 of QITE with starting energy -98.57203477843082
Iteration 12 of QITE with starting energy -98.5720479128784
Iteration 13 of QITE with starting energy -98.57205736692381
Iteration 14 of QITE with starting energy -98.57206418990326
Iteration 15 of QITE with starting energy -98.57206912952093
Iteration 16 of QITE with starting energy -98.5720727188785
Iteration 17 of QITE with starting en

-98.57208287769595

### 5. Recomputing the total energy of the system

In order to assess the impact of quantum workflows applied to one or several fragments on the total energy of the system, the helper class provides a `mi_summation` method. This class recomputes the total energy of the system, using the reference values obtained classically by QEMIST Cloud, or instead using the values obtained by your quantum workflows for the fragments you specify. This summation was described by the formulas in the first section of this notebook.

If no argument is passed to `mi_summation`, then only reference values are used in the summation, which should then be in agreement (up to machine precision) with `fno_fragments.e_tot`, originally read from the QEMIST Cloud result blob.

Otherwise, passing a dictionary using fragment labels as keys and the energy as the corresponding values does the trick. For these fragments, MP2 correction is automatically added, and a new total energy is computed.

The difference between this new total energy and the reference total energy gives you a measure of the accuracy and the impact of combining quantum workflows with classical calculations performed with MI-FNO.

In [8]:
e = fno_fragments.mi_summation({"(1,)": qite.final_energy})
print(f"Reconstructed energy: {e}\nMIFNO QEMIST Cloud energy: {fno_fragments.e_tot}\nDifference: {abs(e-fno_fragments.e_tot)}")

Reconstructed energy: -98.59655271223812
MIFNO QEMIST Cloud energy: -98.59655271223816
Difference: 4.263256414560601e-14


## Closing words

This feature allows us to bring together the state-of-the-art MI-FNO problem decomposition schemes available at scale in QEMIST cloud, and quantum workflows written with Tangelo. It facilitates exploring larger, more industrially-relevant use cases with a combination of classical and quantum workflows of your choice, on subproblems that are more ammenable to current devices. 

For now, this exploration is manual, as we believe the current state of the field does not benefit from a fully automated and streamlined platform... yet! In the future, platforms such as QEMIST Cloud will be able to directly use Tangelo to run entire workflows on cloud infrastructure, using both quantum and classical devices.

Maybe your discoveries can contribute to advancing the field, and bring that future a bit sooner. 

What will you do with Tangelo?